In [16]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys

In [3]:
log_level = 0

In [4]:
graylog_con={
        "user":"merger",
        "password":"pc@pM8rger",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [5]:
api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [6]:
import datetime
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'

In [12]:
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [13]:
gres

u'{"query":"protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:\\"a2:2a:0c:40\\"","built_query":"{\\"from\\":0,\\"size\\":150,\\"query\\":{\\"bool\\":{\\"must\\":{\\"query_string\\":{\\"query\\":\\"protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:\\\\\\"a2:2a:0c:40\\\\\\"\\",\\"allow_leading_wildcard\\":false}},\\"filter\\":{\\"bool\\":{\\"must\\":{\\"range\\":{\\"timestamp\\":{\\"from\\":\\"2018-01-12 04:00:00.000\\",\\"to\\":\\"2018-01-12 11:00:00.000\\",\\"include_lower\\":true,\\"include_upper\\":true}}}}}}},\\"sort\\":[{\\"timestamp\\":{\\"order\\":\\"desc\\"}}]}","used_indices":[{"index_name":"graylog_41","begin":"1970-01-01T00:00:00.000Z","end":"1970-01-01T00:00:00.000Z","calculated_at":"2018-01-22T18:31:43.745Z","took_ms":0},{"index_name":"graylog_36","begin":"2018-01-11T20:58:35.461Z","end":"2018-01-14T00:41:28.217Z","calculated_at":"2018-01-14T00:43:34.379Z","took_ms":4767}],

In [9]:
def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

In [10]:
def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)

        return outputResult

In [17]:
pgres=LoadGraylogResult(gres)

Graylog search found 1 messages
Graylog search load 1 messages


In [18]:
pgres[0]['e212_imsi']

u'425191201068102'